#Importing the necessary libraries import numpy as np import tensorflow
as tf from tensorflow import keras from keras.models import Sequential
import pandas as pd from matplotlib import pyplot as plt %matplotlib
inline import os import cv2 from keras.layers import Dense, Conv2D,
MaxPooling2D, Flatten,GlobalAveragePooling2D from PIL import Image from
sklearn.preprocessing import LabelEncoder from sklearn.metrics import
confusion_matrix, classification_report

# Defining the paths to your image and csv folders

train_val_dir = “/content/C:\_valdrive/MyDrive/charts/train_val”
test_dir = “/content/C:/MyDrive/charts/test” train_path_labels =
“/content/C:\_val.csv” train_val_labels = pd.read_csv(train_path_labels)

# loading the training dataset in numpy array

images = \[\] labels = \[\] for filename in os.listdir(train_val_dir):
if filename.endswith(‘.png’): \# Loading the images and resize them to
(128, 128) with 3 color channels img =
cv2.imread(os.path.join(train_val_dir, filename)) img = cv2.resize(img,
(128, 128)) img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # img = Image.open(os.path.join(train_val_dir, filename))
    img_array = np.array(img)
    # Appending the array to the list of images
    images.append(img_array)
    labels.append(filename)

# Converting the string labels to numerical labels

le = LabelEncoder() labels = le.fit_transform(labels)

# Converting the lists to NumPy arrays

images = np.array(images) labels = np.array(labels) \# Save the arrays
in NumPy format np.save(‘x_train.npy’, images) np.save(‘y_train.npy’,
labels) x_train = np.load(‘x_train.npy’) y_train =
np.load(‘y_train.npy’)

x_train.shape x_train\[:5\]

# loading test dataset in numpy array

images = \[\] labels = \[\] for filename in os.listdir(test_dir): if
filename.endswith(‘.png’): img = cv2.imread(os.path.join(test_dir,
filename)) img = cv2.resize(img, (128, 128)) img = cv2.cvtColor(img,
cv2.COLOR_BGR2RGB) img_array = np.array(img) images.append(img_array)
labels.append(filename)

# Converting the string labels to numerical labels

le = LabelEncoder() labels = le.fit_transform(labels) images =
np.array(images) labels = np.array(labels)

# Saving the arrays in NumPy format

np.save(‘x_test.npy’, images) np.save(‘y_test.npy’, labels) x_test =
np.load(‘x_test.npy’) y_test = np.load(‘y_test.npy’)

# Converting the lists to NumPy array

image_classes = \[‘line’, ‘dot_line’, ‘hbar_categorical’,
‘vbar_categorical’, ‘pie’\] image_classes\[0\] \# map the categories to
the labels array i.e y_train label_map = {‘line’: 0, ‘dot_line’: 1,
‘hbar_categorical’: 2, ‘vbar_categorical’: 3, ‘pie’: 4} y_train =
np.array(\[label_map\[label\] for label in train_val_labels\[‘type’\]\])

y_train x_train=x_train /255 x_test=x_train /255 y_train_index =
train_val_labels\[‘image_index’\] y_train_type =
train_val_labels\[‘type’\] y_train_type\[:5\]

# Splitting the training images and labels into training and validation sets

from sklearn.model_selection import train_test_split x_train, x_test,
y_train, y_test = train_test_split(x_train, y_train, test_size=0.2,
random_state=42) cnn_model = Sequential(\[ Conv2D(filters=16
,kernel_size=(3,3), activation=‘relu’, input_shape=(128,128,3)),
MaxPooling2D(pool_size=(2,2)), Conv2D(32, (3,3), activation=‘relu’),
MaxPooling2D(pool_size=(2,2)), Conv2D(64, (3,3), activation=‘relu’),
MaxPooling2D(pool_size=(2,2)), Flatten(), Dense(128, activation=‘relu’),
Dense(5, activation=‘softmax’)\])

# Compiling the model

cnn_model.compile(optimizer=‘adam’,
loss=‘sparse_categorical_crossentropy’, metrics=\[‘accuracy’\])

# Training the model

history = cnn_model.fit(x_train, y_train, batch_size=1000,
epochs=50,validation_data=(x_test, y_test)) \# Plot the obtained loss
plt.plot(history.history\[‘loss’\])
plt.plot(history.history\[‘val_loss’\]) plt.title(‘Model Loss’)
plt.ylabel(‘Loss’) plt.xlabel(‘Epoch’) plt.legend(\[‘Train’,
‘Validation’\], loc=‘upper right’) plt.show()
cnn_model.evaluate(x_test,y_test) def image_sample(x, y, index):
plt.figure(figsize = (10,2)) plt.imshow(x\[index\])

# image_label = train_val_labels.iloc\[index\]\[‘type’\]

# plt.xlabel(image_label)

plt.xlabel(image_classes\[y\[index\]\]) image_sample(x_test,y_test,1)
image_sample(x_test,y_test,50) image_sample(x_test,y_test,25) y_pred =
cnn_model.predict(x_test) y_pred\[:5\] y_classes = \[np.argmax(element)
for element in y_pred\] y_classes\[:5\]

y_test\[:5\]

# test actual and predicted

# image_sample(x_test,y_test,1) #actual

# image_classes\[y_classes\[1\]\] #predicted

# image_sample(x_test,y_test,10) #actual

# image_classes\[y_classes\[10\]\] #predicted

image_sample(x_test,y_test,15) #actual image_classes\[y_classes\[15\]\]
#predicted

print(“classification report: ”,
classification_report(y_test,y_classes))

# Generating the confusion matrix

conf_mat = confusion_matrix(y_test, y_classes) print(‘Confusion
Matrix:’) print(conf_mat)

#Plotting the confusion matrix import seaborn as sn plt.figure(figsize =
(10,10)) sn.heatmap(conf_mat,annot=True,fmt=‘d’) plt.xlabel(‘Predicted’)
plt.ylabel(‘Actual’) from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Loading the pre-trained model

vgg16_model = VGG16(weights=‘imagenet’, include_top=False,
input_shape=(224, 224, 3))

# Replacing the final classification layer with a new layer

x = vgg16_model.output x = GlobalAveragePooling2D()(x) x = Dense(128,
activation=‘relu’)(x) predictions = Dense(5, activation=‘softmax’)(x)
pt_model = tf.keras.Model(inputs=vgg16_model.input, outputs=predictions)
for layer in pt_model.layers: layer.trainable = False

# Printing the summary of the model architecture

pt_model.summary() train_datagen = ImageDataGenerator( rescale=1./255,
rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
shear_range=0.2, zoom_range=0.2, horizontal_flip=True,
fill_mode=‘nearest’) test_datagen = ImageDataGenerator(rescale=1./255)

# flow method generates batches of augmented data

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
test_generator = train_datagen.flow(x_test, y_test, batch_size=32)
plt.plot(history.history\[‘loss’\])
plt.plot(history.history\[‘val_loss’\]) plt.title(‘Model Loss’)
plt.ylabel(‘Loss’) plt.xlabel(‘Epoch’) plt.legend(\[‘Train’,
‘Validation’\], loc=‘upper right’) plt.show()